## Token processing

#### BPE (Byte Pair Encoding) 모델 훈련 방법

In [14]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

# BPE 토크나이저 초기화
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()

# 트레이너 설정 및 학습
trainer = BpeTrainer(special_tokens=["[UNK]"], vocab_size=50)
training_data = ["low lower lowest", "new newer newest"]
tokenizer.train_from_iterator(training_data, trainer)

# 토크나이징 결과
result = tokenizer.encode("low lower lowest")
print(f"토큰: {result.tokens}")
print(f"토큰 ID: {result.ids}")


토큰: ['low', 'lower', 'lowest']
토큰 ID: [15, 16, 17]


#### 사전 학습된 BPE 모델 사용방법

In [ ]:
from transformers import GPT2Tokenizer # 많이 사용되는 BPE 모델

tokenizer = GPT2Tokenizer.from_pretrained('gpt2') # GPT-2 BPE 토크나이저

test_text = "Artificial intelligence is transforming our world"
tokens = tokenizer.tokenize(test_text)
token_ids = tokenizer.encode(test_text)

print(f"원본: '{test_text}'")
print(f"토큰: {tokens}")
print(f"토큰 ID: {token_ids}")

examples = ["AI is cool", "supercalifragilisticexpialidocious", "COVID-19"]
for text in examples:
    tokens = tokenizer.tokenize(text)
    token_ids = tokenizer.encode(text)
    print(f"'{text}'")
    print(f"  토큰: {tokens}")
    print(f"  ID: {token_ids}")
    print()

원본: 'Artificial intelligence is transforming our world'
토큰: ['Art', 'ificial', 'Ġintelligence', 'Ġis', 'Ġtransforming', 'Ġour', 'Ġworld']
토큰 ID: [8001, 9542, 4430, 318, 25449, 674, 995]
'AI is cool'
  토큰: ['AI', 'Ġis', 'Ġcool']
  ID: [20185, 318, 3608]

'supercalifragilisticexpialidocious'
  토큰: ['super', 'cal', 'if', 'rag', 'il', 'ist', 'ice', 'xp', 'ial', 'id', 'ocious']
  ID: [16668, 9948, 361, 22562, 346, 396, 501, 42372, 498, 312, 32346]

'COVID-19'
  토큰: ['CO', 'VID', '-', '19']
  ID: [8220, 11008, 12, 1129]



: 

#### BPE 직접 구현해보기

In [22]:
import re
from collections import defaultdict, Counter

def get_word_tokens(text): # 텍스트를 단어 단위로 분리하고 문자 단위로 토큰화
	words = text.lower().split()
	word_tokens = {}
	for word in words:
		tokens = list(word) + ['</w>'] # 각 문자를 분리하고 단어 끝에 </w> 추가
		word_tokens[' '.join(tokens)] = word_tokens.get(' '.join(tokens), 0) + 1
	return word_tokens

def get_bi_grams(word_tokens): # 모든 바이그램과 빈도수 계산
	bi_grams = defaultdict(int)
	for word, freq in word_tokens.items():
		symbols = word.split()
		for i in range(len(symbols) - 1):
			bi_grams[(symbols[i], symbols[i+1])] += freq
	return bi_grams

def merge_symbols(pair, word_tokens): # 선택된 바이그램을 병합
	new_word_tokens = {}
	bi_gram = ' '.join(pair)
	replacement = ''.join(pair)
	
	for word in word_tokens:
		new_word = word.replace(bi_gram, replacement)
		new_word_tokens[new_word] = word_tokens[word]
	return new_word_tokens

# BPE 예시 - 더 명확한 결과를 위한 텍스트
text = "low lower lowest"
print(f"원본 텍스트: {text}")

# 1. 초기 토큰화
word_tokens = get_word_tokens(text)
print(f"초기 토큰화: {word_tokens}")
print(f"초기 어휘 크기: {len(set(' '.join(word_tokens.keys()).split()))}")

# 2. BPE 학습 과정 상세히 보기
for i in range(2):  # 더 많은 병합 과정 관찰
	bi_grams = get_bi_grams(word_tokens)
	if not bi_grams:
		break
	
	# 모든 바이그램과 빈도 출력
	print(f"\n=== 반복 {i+1} ===")
	print(f"현재 바이그램 빈도: {dict(sorted(bi_grams.items(), key=lambda x: x[1], reverse=True))}")
	
	# 가장 빈도가 높은 바이그램 선택
	best_pair = max(bi_grams, key=bi_grams.get)
	print(f"선택된 바이그램: '{best_pair}' (빈도: {bi_grams[best_pair]})")

	# 바이그램 병합
	word_tokens = merge_symbols(best_pair, word_tokens)
	print(f"병합 후: {word_tokens}")

final_vocab = set(' '.join(word_tokens.keys()).split())
print(f"최종 BPE 어휘: {sorted(final_vocab)}")
print(f"어휘 크기 변화: 초기 {len(set('low lower lowest new newer newest'.replace(' ', '') + '</w>'))}개 → 최종 {len(final_vocab)}개")

원본 텍스트: low lower lowest
초기 토큰화: {'l o w </w>': 1, 'l o w e r </w>': 1, 'l o w e s t </w>': 1}
초기 어휘 크기: 8

=== 반복 1 ===
현재 바이그램 빈도: {('l', 'o'): 3, ('o', 'w'): 3, ('w', 'e'): 2, ('w', '</w>'): 1, ('e', 'r'): 1, ('r', '</w>'): 1, ('e', 's'): 1, ('s', 't'): 1, ('t', '</w>'): 1}
선택된 바이그램: '('l', 'o')' (빈도: 3)
병합 후: {'lo w </w>': 1, 'lo w e r </w>': 1, 'lo w e s t </w>': 1}

=== 반복 2 ===
현재 바이그램 빈도: {('lo', 'w'): 3, ('w', 'e'): 2, ('w', '</w>'): 1, ('e', 'r'): 1, ('r', '</w>'): 1, ('e', 's'): 1, ('s', 't'): 1, ('t', '</w>'): 1}
선택된 바이그램: '('lo', 'w')' (빈도: 3)
병합 후: {'low </w>': 1, 'low e r </w>': 1, 'low e s t </w>': 1}
최종 BPE 어휘: ['</w>', 'e', 'low', 'r', 's', 't']
어휘 크기 변화: 초기 11개 → 최종 6개


#### Tokenization using BERT

In [2]:
from transformers import BertTokenizer, BertModel
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

example_sen = (
    """
    The United States and Russia sought to lower the temperature in a
    heated standoff over Ukraine,even as they reported no breakthroughs
    in high-stakes talks on Friday aimed at preventing a feared Russian invasion
    """
)
print(bert_tokenizer.tokenize(example_sen))

['the', 'united', 'states', 'and', 'russia', 'sought', 'to', 'lower', 'the', 'temperature', 'in', 'a', 'heated', 'stand', '##off', 'over', 'ukraine', ',', 'even', 'as', 'they', 'reported', 'no', 'breakthrough', '##s', 'in', 'high', '-', 'stakes', 'talks', 'on', 'friday', 'aimed', 'at', 'preventing', 'a', 'feared', 'russian', 'invasion']
